In [2]:
import pandas as pd
from apyori import apriori

In [9]:
df = pd.read_csv('/home/marcela/projetos/models_ml2/apriori/compras_mercado/mercado.csv', header=None)

In [10]:
df

,0,1,2,3
0,leite,cerveja,pão,manteiga
1,café,pão,manteiga,NaN
2,café,pão,manteiga,NaN
3,leite,café,pão,manteiga
4,cerveja,NaN,NaN,NaN
5,manteiga,NaN,NaN,NaN
6,pão,NaN,NaN,NaN
7,feijão,NaN,NaN,NaN
8,arroz,feijão,NaN,NaN
9,arroz,NaN,NaN,NaN
